In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use("seaborn-colorblind")
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import xarray as xr
from lib.thermo import liquid_water_temperature, get_dz, mass_integrate
from lib.advection import material_derivative

stat = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc")
fields_3d = xr.open_mfdataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/*.nc")
fields_2d = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc")

# for some reason the time values are all scrambled up.
fields_2d = fields_2d.isel(time=fields_2d.time.values.argsort())
data = xr.merge((fields_2d, fields_3d), join='inner').isel(y=slice(24, 40))

p = stat.p
rho = stat.RHO[0]
dz = get_dz(rho.z)
data  = data.assign(
    sl=liquid_water_temperature(data.TABS, data.QN, data.QP),
    qt=data.QV + data.QN
)

In [ ]:
def precip_from_fsl(fsl, shf):
    return ((fsl-data.QRAD) *rho * dz).sum('z') * 1004/2.51e6 - shf/2.51e6 *86400

def precip_from_fqt(fqt, lhf):
    return -(fqt * rho * dz).sum('z').compute()/1000 + lhf/2.51e6*86400

In [ ]:
dqt = material_derivative(data.U, data.V, data.W, data.qt)*86400 + data.qt.diff('time')/.125
prec_pred = precip_from_fqt(dqt, data.LHF)

dsl = material_derivative(data.U, data.V, data.W, data.sl)*86400 + data.sl.diff('time')/.125
prec_pred_t = precip_from_fsl(dsl, data.SHF)


precip = xr.Dataset({'truth': data.Prec, 'prec_budg': prec_pred, 'prec_budg_t': prec_pred_t})

In [ ]:
import torch
from lib.models.torch import wrap
from toolz import pipe
from toolz.curried import valmap
import torch
from torch.autograd import Variable

def wrap(torch_model):
    def fun(*args):
        torch_args = [pipe(x, _dataset_to_dict,
                           valmap(torch.FloatTensor),
                           valmap(Variable))
                      for x in args]
        y = torch_model(*torch_args)
        y = valmap(lambda x: x.cpu().data.numpy(), y)

        # get coords from inputs
        x = args[0]

        return xr.Dataset(valmap(
            lambda arr: xr.DataArray(arr, coords=x.coords, dims=x.dims),
            y
        ))

    return fun

def _dataset_to_dict(ds: xr.Dataset):
    return {key: prepare_array(ds[key]) for key in ds.data_vars}

def prepare_array(x):

    return x.transpose('batch', 'time', 'z').values


calc_src =  wrap(torch.load("../data/ml/ngaqua/model.1.torch").rhs)

In [ ]:
din = data[['qt', 'sl']].stack(batch=['x', 'y'])


src = calc_src(din).unstack('batch')

In [ ]:
prec_nn_t = precip_from_fsl(src.sl, data.SHF)

prec_nn = precip_from_fqt(src.qt.isel(z=slice(0,-15)), data.LHF)

precip = precip.assign(nn=prec_nn, nnt=prec_nn_t).compute()

In [ ]:
%%opts Curve[width=500](color=Cycle(['k', 'b', 'light green', 'brown', 'c']))
ds = hv.Dataset(precip.mean(['x','time']).to_array().to_dataset(name="Precip"))
ds.to.curve("y").overlay().redim.unit(Precip="mm/day")

From this it looks like the precipitation estimated from the neural networks temperature equation gives the best match. It does even better than the terms predicted from the actual budgets. For some reason the moisture equation significantly over-estimates the precip. It could just be the model has not trained sufficiently, or perhaps this is just a random effect of the initialization of the weights in the training procedure.